In [1]:
import onnxruntime as ort

# get execution provider
providers = ort.get_available_providers()
providers

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

In [2]:
%env LD_LIBRARY_PATH=/home/dnth/mambaforge-pypy3/envs/ram-onnx-inference/lib/python3.11/site-packages/tensorrt_libs:/home/dnth/mambaforge-pypy3/envs/ram-onnx-inference/lib:$LD_LIBRARY_PATH

env: LD_LIBRARY_PATH=/home/dnth/mambaforge-pypy3/envs/ram-onnx-inference/lib/python3.11/site-packages/tensorrt_libs:/home/dnth/mambaforge-pypy3/envs/ram-onnx-inference/lib:$LD_LIBRARY_PATH


In [3]:

from PIL import Image
from urllib.request import urlopen

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

img


import numpy as np
from PIL import Image
def transforms_numpy(image: Image.Image):
    image = image.convert('RGB')
    image = image.resize((448, 448), Image.BICUBIC)
    img_numpy = np.array(image).astype(np.float32) / 255.0
    img_numpy = img_numpy.transpose(2, 0, 1)

    mean = np.array([0.485, 0.456, 0.406]).reshape(-1, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(-1, 1, 1)
    img_numpy = (img_numpy - mean) / std
    img_numpy = np.expand_dims(img_numpy, axis=0)
    img_numpy = img_numpy.astype(np.float32)

    return img_numpy


import onnxruntime as ort

providers = ['TensorrtExecutionProvider']
session = ort.InferenceSession('eva02_large_patch14_448.onnx', providers=providers)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

output = session.run([output_name], {input_name: transforms_numpy(img)})
output

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider']
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************


2024-09-10 11:19:52.296452091 [E:onnxruntime:Default, provider_bridge_ort.cc:1978 TryGetProviderInfo_TensorRT] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1637 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libcudnn.so.9: cannot open shared object file: No such file or directory



[array([[-1.77464217e-01, -1.13817573e-01, -8.76026601e-02,
          8.69853124e-02,  7.27204144e-01, -4.62081969e-01,
         -1.76158428e-01, -3.96329612e-01, -3.04876298e-01,
         -3.45760286e-01, -4.94259745e-01, -1.62426859e-01,
         -1.31224275e-01, -1.17028162e-01,  1.04696602e-02,
         -1.04521327e-01, -2.02701300e-01, -6.22882172e-02,
         -1.12694219e-01, -6.03306115e-01, -4.64288175e-01,
          2.93670595e-02, -1.33947253e-01,  2.18217090e-01,
         -5.31307459e-01,  4.84423935e-02, -3.59963179e-02,
         -5.88736832e-01,  3.68650526e-01, -4.85611260e-01,
         -2.56656110e-01, -4.01910871e-01,  5.98749518e-03,
         -9.85038280e-02, -8.46037418e-02, -2.06972569e-01,
         -2.61949062e-01, -3.54980081e-02,  1.25174671e-01,
         -1.63902849e-01,  2.21279696e-01, -1.22567803e-01,
         -5.68985939e-04, -1.88159660e-01,  3.55500519e-01,
         -1.33886173e-01,  3.23494673e-01, -1.90455168e-02,
         -6.46073669e-02, -3.13407779e-0

In [ ]:
import time

num_images = 10
start = time.perf_counter()
for i in range(num_images):
    output = session.run([output_name], {input_name: transforms_numpy(img)})
end = time.perf_counter()
time_taken = end - start

ms_per_image = time_taken / num_images * 1000
fps = num_images / time_taken

print(f"Onnxruntime TensorRT: {ms_per_image:.3f} ms per image, FPS: {fps:.2f}")